In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

from bs4 import BeautifulSoup

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-3.3.0               |           py36_0         747 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [2]:
bham_wiki_url = requests.get('https://en.wikipedia.org/wiki/B_postcode_area').text # This will ping a website and return the HTML of the website

soup = BeautifulSoup(bham_wiki_url,'lxml') # This creates a BeautifulSoup object via BeautifulSoup function. This package is for parsing HTML and XML documents
#print(soup.prettify()) # 'Prettify' will let us view how the tags are nested in the document
table = soup.find_all('table')[1]
#print(table.prettify())
df1 = pd.read_html(str(table))
df1 = df1[0]
df1.sample(10)

,Postcode district,Post town,Coverage,Local authority area
15,B16,BIRMINGHAM,Ladywood,Birmingham
3,B4,BIRMINGHAM,"Birmingham City Centre, Corporation Street (no...",Birmingham
2,B3,BIRMINGHAM,"Birmingham City Centre, Newhall Street",Birmingham
32,B34,BIRMINGHAM,"Shard End, Buckland End",Birmingham
46,B50,ALCESTER,Bidford-on-Avon,Stratford-on-Avon
33,B35,BIRMINGHAM,Castle Vale,Birmingham
58,B71,WEST BROMWICH,West Bromwich (north),Sandwell
50,B63,HALESOWEN,"Halesowen (west and town centre), Hayley Green...","Dudley, Bromsgrove"
52,B65,ROWLEY REGIS,"Rowley Regis, Blackheath","Birmingham, Sandwell"
67,B80,STUDLEY,Studley,Stratford-on-Avon


In [3]:
#Here we check the data and try to correct missing values or irrelevant rows. The last row is non-geographical hence we will drop the row
print('Columns in bham dataframe with null values:\n', df1.isnull().sum())
print('-'*10)
print(df1.info())

Columns in bham dataframe with null values:
 Postcode district       0
Post town               0
Coverage                1
Local authority area    0
dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 4 columns):
Postcode district       78 non-null object
Post town               78 non-null object
Coverage                77 non-null object
Local authority area    78 non-null object
dtypes: object(4)
memory usage: 2.5+ KB
None


In [4]:
df1.drop(77, axis=0, inplace=True) #Comment line if already run once
df1.replace('Halesowen (west and town centre), Hayley Green, Hasbury','Halesowen town centre', inplace=True) #Comment line if already run once

print('Columns in bham dataframe with null values:\n', df1.isnull().sum())
print('-'*10)
print(df1.info())

Columns in bham dataframe with null values:
 Postcode district       0
Post town               0
Coverage                0
Local authority area    0
dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 76
Data columns (total 4 columns):
Postcode district       77 non-null object
Post town               77 non-null object
Coverage                77 non-null object
Local authority area    77 non-null object
dtypes: object(4)
memory usage: 3.0+ KB
None


In [6]:
!conda install -c conda-forge geopy --yes # comment this line if not needed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [7]:
import geopy.geocoders # import geocoder

In [8]:
from geopy.exc import GeocoderTimedOut
from geopy.exc import GeocoderUnavailable

def do_geocode(address):
    geopy = Nominatim(user_agent="bham_explorer")
    try:
        return geopy.geocode(address, timeout=1)
    except GeocoderUnavailable:
        return do_geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [9]:
address = 'Smethwick B66, UK'

geolocator = Nominatim(user_agent="bham_explorer")
location = do_geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Smethwick B66, UK are 52.4968157, -1.9739174.


In [10]:
b96_map = folium.Map(location=[latitude, longitude], zoom_start=15)

b96_map

In [11]:
df1['Address'] = df1['Coverage'] +', '+ df1['Postcode district']
df1

,Postcode district,Post town,Coverage,Local authority area,Address
0,B1,BIRMINGHAM,"Birmingham City Centre, Broad Street (east)",Birmingham,"Birmingham City Centre, Broad Street (east), B1"
1,B2,BIRMINGHAM,"Birmingham City Centre, New Street",Birmingham,"Birmingham City Centre, New Street, B2"
2,B3,BIRMINGHAM,"Birmingham City Centre, Newhall Street",Birmingham,"Birmingham City Centre, Newhall Street, B3"
3,B4,BIRMINGHAM,"Birmingham City Centre, Corporation Street (no...",Birmingham,"Birmingham City Centre, Corporation Street (no..."
4,B5,BIRMINGHAM,"Digbeth, Highgate, Lee Bank",Birmingham,"Digbeth, Highgate, Lee Bank, B5"
5,B6,BIRMINGHAM,"Aston, Witton",Birmingham,"Aston, Witton, B6"
6,B7,BIRMINGHAM,Nechells,Birmingham,"Nechells, B7"
7,B8,BIRMINGHAM,"Washwood Heath, Ward End, Saltley",Birmingham,"Washwood Heath, Ward End, Saltley, B8"
8,B9,BIRMINGHAM,"Bordesley Green, Bordesley",Birmingham,"Bordesley Green, Bordesley, B9"
9,B10,BIRMINGHAM,Small Heath,Birmingham,"Small Heath, B10"


In [12]:
#df1['location'] = df1['Address'].apply(geolocator.geocode)
df2 = df1.copy(deep=True)
df2 = df2.iloc[:30]
df2.shape

(30, 5)

In [13]:
def getLatLngByAddress(postcode, neighbourhood, address):
    
    lat_lng_list=[]
    for pc, nbh, address in zip(postcode, neighbourhood, address):
        print(pc)
        
        #geocoders request for latitude and longitude coordinates
        try:
            print(address)
            location = do_geocode(address)
            latitude = location.latitude
            longitude = location.longitude
        except:
            try:
                print(nbh)
                location = do_geocode(nbh)
                latitude = location.latitude
                longitude = location.longitude
            except:
                try:
                    x = nbh.split(", ")
                    print(x)
                    location = do_geocode(x[0] + ', UK')
                    latitude = location.latitude
                    longitude = location.longitude
                except:
                    try:
                        print(x[1:])
                        location = do_geocode(x[1] + ', UK')
                        latitude = location.latitude
                        longitude = location.longitude
                    except:
                        print(x[2:])
                        location = do_geocode(x[2] + ', UK')
                        latitude = location.latitude
                        longitude = location.longitude

            
        
        lat_lng_list.append([(pc, 
                              nbh, 
                              latitude, 
                              longitude)])
    
    #location_df = pd.DataFrame(data=lat_lng_list)
    #location_df.columns = ['Postcode',
    #                      'Neighbourhood',
    #                      'Latitude',
    #                      'Longitude']
    
    return(lat_lng_list)

In [14]:
lat_lng_list = getLatLngByAddress(postcode=df2['Postcode district'], neighbourhood=df2['Coverage'], address=df2['Address'])

B1
Birmingham City Centre, Broad Street (east), B1
Birmingham City Centre, Broad Street (east)
['Birmingham City Centre', 'Broad Street (east)']
B2
Birmingham City Centre, New Street, B2
Birmingham City Centre, New Street
B3
Birmingham City Centre, Newhall Street, B3
Birmingham City Centre, Newhall Street
B4
Birmingham City Centre, Corporation Street (north), B4
Birmingham City Centre, Corporation Street (north)
['Birmingham City Centre', 'Corporation Street (north)']
B5
Digbeth, Highgate, Lee Bank, B5
Digbeth, Highgate, Lee Bank
['Digbeth', 'Highgate', 'Lee Bank']
B6
Aston, Witton, B6
B7
Nechells, B7
B8
Washwood Heath, Ward End, Saltley, B8
B9
Bordesley Green, Bordesley, B9
B10
Small Heath, B10
B11
Sparkhill, Sparkbrook, Tyseley, B11
Sparkhill, Sparkbrook, Tyseley
['Sparkhill', 'Sparkbrook', 'Tyseley']
B12
Balsall Heath, Sparkbrook, Highgate, B12
Balsall Heath, Sparkbrook, Highgate
['Balsall Heath', 'Sparkbrook', 'Highgate']
B13
Moseley, Billesley, B13
B14
Kings Heath, Yardley Wood, D

In [15]:
l1 = np.array(lat_lng_list)
l1.shape
print(np.info(l1))
print('-'*10)
print(l1)

class:  ndarray
shape:  (30, 1, 4)
strides:  (1120, 1120, 280)
itemsize:  280
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x55771c4ca500
byteorder:  little
byteswap:  False
type: <U70
None
----------
[[['B1' 'Birmingham City Centre, Broad Street (east)' '52.4775396'
   '-1.894053']]

 [['B2' 'Birmingham City Centre, New Street' '52.4792602' '-1.8999756']]

 [['B3' 'Birmingham City Centre, Newhall Street' '52.4832071'
   '-1.9054204']]

 [['B4' 'Birmingham City Centre, Corporation Street (north)'
   '52.4775396' '-1.894053']]

 [['B5' 'Digbeth, Highgate, Lee Bank' '52.4734488' '-1.8871192']]

 [['B6' 'Aston, Witton' '52.506768' '-1.8806006']]

 [['B7' 'Nechells' '52.5023095' '-1.8605038']]

 [['B8' 'Washwood Heath, Ward End, Saltley' '52.4865451' '-1.8330255']]

 [['B9' 'Bordesley Green, Bordesley' '52.4778205' '-1.8487612']]

 [['B10' 'Small Heath' '52.4681262' '-1.8513874']]

 [['B11' 'Sparkhill, Sparkbrook, Tyseley' '52.4533095' '-1.8675068']]

 [['B12' 'Balsall He

In [16]:
lat_list = []
lng_list = []
for i in range(30):
    lat_list.append(l1[i][0][2])
    lng_list.append(l1[i][0][3])
    i += 1
    
print('Latitude: \n', lat_list)
print('Longitude: \n', lng_list)
#len(lng_list)

Latitude: 
 ['52.4775396', '52.4792602', '52.4832071', '52.4775396', '52.4734488', '52.506768', '52.5023095', '52.4865451', '52.4778205', '52.4681262', '52.4533095', '52.4572246', '52.4347571', '52.4116475', '52.46882645', '52.4805954', '52.45960005', '52.4968772', '52.4996276', '52.5130035', '52.50767645', '52.5232514', '52.52939455', '52.4709562', '52.4616984', '52.4464213', '52.4285007', '52.440222', '52.4273865', '52.4082313']
Longitude: 
 ['-1.894053', '-1.8999756', '-1.9054204', '-1.894053', '-1.8871192', '-1.8806006', '-1.8605038', '-1.8330255', '-1.8487612', '-1.8513874', '-1.8675068', '-1.8832067', '-1.87774549212102', '-1.8655917', '-1.92343540104981', '-1.918506', '-1.96040213627551', '-1.9380313', '-1.9068745', '-1.9067598', '-1.93312430441457', '-1.8395109', '-1.83081205238147', '-1.8159436', '-1.7828480818756', '-1.8210145', '-1.840403', '-1.9382689', '-1.9266395', '-1.965252']


In [17]:
location_df = pd.DataFrame({ "Latitude" : lat_list, "Longitude" : lng_list})
#df = [df2 , location_df] 
df = df2.join(location_df)
df[['Latitude', 'Longitude']].astype('float64').dtypes

Latitude     float64
Longitude    float64
dtype: object

In [18]:
df.drop(['Post town','Local authority area','Address'], axis=1, inplace=True)
df

,Postcode district,Coverage,Latitude,Longitude
0,B1,"Birmingham City Centre, Broad Street (east)",52.4775396,-1.894053
1,B2,"Birmingham City Centre, New Street",52.4792602,-1.8999756
2,B3,"Birmingham City Centre, Newhall Street",52.4832071,-1.9054204
3,B4,"Birmingham City Centre, Corporation Street (no...",52.4775396,-1.894053
4,B5,"Digbeth, Highgate, Lee Bank",52.4734488,-1.8871192
5,B6,"Aston, Witton",52.506768,-1.8806006
6,B7,Nechells,52.5023095,-1.8605038
7,B8,"Washwood Heath, Ward End, Saltley",52.4865451,-1.8330255
8,B9,"Bordesley Green, Bordesley",52.4778205,-1.8487612
9,B10,Small Heath,52.4681262,-1.8513874


In [19]:
# create map of Birmingham UK using latitude and longitude values
latitude = 52.4862
longitude = -1.8904
map_bham = folium.Map(location=[latitude, longitude], zoom_start=12.3)

# add markers to map
for lat, lng, pc, nbh in zip(df['Latitude'], df['Longitude'], df['Postcode district'], df['Coverage']):
    label = 'Postcode: {}\n, Area: {}\n'.format(pc, nbh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bham)  
    
map_bham

In [20]:
postal_district_urls = ['https://en.wikipedia.org/wiki/E_postcode_area', 'https://en.wikipedia.org/wiki/EC_postcode_area', 'https://en.wikipedia.org/wiki/N_postcode_area', 'https://en.wikipedia.org/wiki/WC_postcode_area', 'https://en.wikipedia.org/wiki/W_postcode_area']

In [36]:
def PostalDataframeScrapeLDN(urls):
    df_list = []
    for i in range(len(urls)):
        ldn_wiki_url = requests.get(urls[i]).text
        soup = BeautifulSoup(ldn_wiki_url,'lxml') # This creates a BeautifulSoup object via BeautifulSoup function. This package is for parsing HTML and XML documents
        table = soup.find_all('table')[1]
        df1 = pd.read_html(str(table))
        df1 = df1[0]
        df_list.append(df1)
    
    df = pd.concat(df_list).reset_index(drop=True)
    return df
df

,Postcode district,Coverage,Latitude,Longitude
0,B1,"Birmingham City Centre, Broad Street (east)",52.4775396,-1.894053
1,B2,"Birmingham City Centre, New Street",52.4792602,-1.8999756
2,B3,"Birmingham City Centre, Newhall Street",52.4832071,-1.9054204
3,B4,"Birmingham City Centre, Corporation Street (no...",52.4775396,-1.894053
4,B5,"Digbeth, Highgate, Lee Bank",52.4734488,-1.8871192
5,B6,"Aston, Witton",52.506768,-1.8806006
6,B7,Nechells,52.5023095,-1.8605038
7,B8,"Washwood Heath, Ward End, Saltley",52.4865451,-1.8330255
8,B9,"Bordesley Green, Bordesley",52.4778205,-1.8487612
9,B10,Small Heath,52.4681262,-1.8513874


In [22]:
df_ldn = PostalDataframeScrapeLDN(urls=postal_district_urls)
df_ldn

,Postcode district,Post town,Coverage,Local authority area
0,E1,LONDON,"Eastern head district: Aldgate (part), Bishops...","Tower Hamlets, Hackney, City of London"
1,E1W,LONDON,"Wapping, St Katharine Docks, Stepney (part), S...",Tower Hamlets
2,E2,LONDON,"Bethnal Green district: Bethnal Green, Haggers...","Tower Hamlets, Hackney"
3,E3,LONDON,"Bow district: Bow, Bow Common, Bromley-by-Bow,...","Tower Hamlets, Newham"
4,E4,LONDON,"Chingford district: Chingford, Sewardstone, Hi...","Waltham Forest, Enfield, Epping Forest (Essex)"
5,E5,LONDON,"Clapton district: Leyton (Part), Upper Clapton...","Hackney, Waltham Forest"
6,E6,LONDON,"East Ham district: East Ham, Beckton, Upton Pa...","Newham, Barking and Dagenham"
7,E7,LONDON,"Forest Gate district: Forest Gate, Leytonstone...","Newham, Waltham Forest"
8,E8,LONDON,"Hackney district: Hackney Central, Dalston, Lo...",Hackney
9,E9,LONDON,"Homerton district: Homerton, Hackney Wick, Sou...","Hackney, Tower Hamlets"


In [23]:
df_ldn.shape

(115, 4)

In [24]:
#Here we check the data and try to correct missing values or irrelevant rows. The last row is non-geographical hence we will drop the row
print('Columns in ldn dataframe with null values:\n', df_ldn.isnull().sum())
print('-'*10)
print(df_ldn.info())

Columns in ldn dataframe with null values:
 Postcode district       0
Post town               0
Coverage                5
Local authority area    0
dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 4 columns):
Postcode district       115 non-null object
Post town               115 non-null object
Coverage                110 non-null object
Local authority area    115 non-null object
dtypes: object(4)
memory usage: 3.7+ KB
None


In [25]:
print('Columns in ldn dataframe with null values:\n', df_ldn.isnull().sum())
print('-'*10)
print(df_ldn.info())
df_ldn

Columns in ldn dataframe with null values:
 Postcode district       0
Post town               0
Coverage                5
Local authority area    0
dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 4 columns):
Postcode district       115 non-null object
Post town               115 non-null object
Coverage                110 non-null object
Local authority area    115 non-null object
dtypes: object(4)
memory usage: 3.7+ KB
None


,Postcode district,Post town,Coverage,Local authority area
0,E1,LONDON,"Eastern head district: Aldgate (part), Bishops...","Tower Hamlets, Hackney, City of London"
1,E1W,LONDON,"Wapping, St Katharine Docks, Stepney (part), S...",Tower Hamlets
2,E2,LONDON,"Bethnal Green district: Bethnal Green, Haggers...","Tower Hamlets, Hackney"
3,E3,LONDON,"Bow district: Bow, Bow Common, Bromley-by-Bow,...","Tower Hamlets, Newham"
4,E4,LONDON,"Chingford district: Chingford, Sewardstone, Hi...","Waltham Forest, Enfield, Epping Forest (Essex)"
5,E5,LONDON,"Clapton district: Leyton (Part), Upper Clapton...","Hackney, Waltham Forest"
6,E6,LONDON,"East Ham district: East Ham, Beckton, Upton Pa...","Newham, Barking and Dagenham"
7,E7,LONDON,"Forest Gate district: Forest Gate, Leytonstone...","Newham, Waltham Forest"
8,E8,LONDON,"Hackney district: Hackney Central, Dalston, Lo...",Hackney
9,E9,LONDON,"Homerton district: Homerton, Hackney Wick, Sou...","Hackney, Tower Hamlets"


In [26]:
df_ldn['Address'] = df_ldn['Coverage'] +', '+ df_ldn['Postcode district']
df2 = df_ldn.copy(deep=True)
df2 = df2.iloc[:30]
df2.reset_index(drop=True, inplace=True)
df2.drop([3, 7, 12, 14, 15, 18, 19, 28, 29], axis=0, inplace=True)
df2

,Postcode district,Post town,Coverage,Local authority area,Address
0,E1,LONDON,"Eastern head district: Aldgate (part), Bishops...","Tower Hamlets, Hackney, City of London","Eastern head district: Aldgate (part), Bishops..."
1,E1W,LONDON,"Wapping, St Katharine Docks, Stepney (part), S...",Tower Hamlets,"Wapping, St Katharine Docks, Stepney (part), S..."
2,E2,LONDON,"Bethnal Green district: Bethnal Green, Haggers...","Tower Hamlets, Hackney","Bethnal Green district: Bethnal Green, Haggers..."
4,E4,LONDON,"Chingford district: Chingford, Sewardstone, Hi...","Waltham Forest, Enfield, Epping Forest (Essex)","Chingford district: Chingford, Sewardstone, Hi..."
5,E5,LONDON,"Clapton district: Leyton (Part), Upper Clapton...","Hackney, Waltham Forest","Clapton district: Leyton (Part), Upper Clapton..."
6,E6,LONDON,"East Ham district: East Ham, Beckton, Upton Pa...","Newham, Barking and Dagenham","East Ham district: East Ham, Beckton, Upton Pa..."
8,E8,LONDON,"Hackney district: Hackney Central, Dalston, Lo...",Hackney,"Hackney district: Hackney Central, Dalston, Lo..."
9,E9,LONDON,"Homerton district: Homerton, Hackney Wick, Sou...","Hackney, Tower Hamlets","Homerton district: Homerton, Hackney Wick, Sou..."
10,E10,LONDON,"Leyton district: Leyton, Temple Mills, Hackney...","Waltham Forest, Hackney","Leyton district: Leyton, Temple Mills, Hackney..."
11,E11,LONDON,"Leytonstone district: Leytonstone, Wanstead, A...","Waltham Forest, Redbridge","Leytonstone district: Leytonstone, Wanstead, A..."


In [27]:
lat_lng_list = getLatLngByAddress(postcode=df2['Postcode district'], neighbourhood=df2['Coverage'], address=df2['Address'])

E1
Eastern head district: Aldgate (part), Bishopsgate (part), Whitechapel, Shoreditch (part), Spitalfields, Shadwell, Stepney, Mile End (part), Portsoken, E1
Eastern head district: Aldgate (part), Bishopsgate (part), Whitechapel, Shoreditch (part), Spitalfields, Shadwell, Stepney, Mile End (part), Portsoken
['Eastern head district: Aldgate (part)', 'Bishopsgate (part)', 'Whitechapel', 'Shoreditch (part)', 'Spitalfields', 'Shadwell', 'Stepney', 'Mile End (part)', 'Portsoken']
['Bishopsgate (part)', 'Whitechapel', 'Shoreditch (part)', 'Spitalfields', 'Shadwell', 'Stepney', 'Mile End (part)', 'Portsoken']
['Whitechapel', 'Shoreditch (part)', 'Spitalfields', 'Shadwell', 'Stepney', 'Mile End (part)', 'Portsoken']
E1W
Wapping, St Katharine Docks, Stepney (part), Shadwell (part), Whitechapel (part), E1W
Wapping, St Katharine Docks, Stepney (part), Shadwell (part), Whitechapel (part)
['Wapping', 'St Katharine Docks', 'Stepney (part)', 'Shadwell (part)', 'Whitechapel (part)']
E2
Bethnal Green d

IndexError: list index out of range

In [28]:
l1 = np.array(lat_lng_list)
l1.shape
print(np.info(l1))
print('-'*25)
print(l1)

class:  ndarray
shape:  (30, 1, 4)
strides:  (1120, 1120, 280)
itemsize:  280
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x55771c548990
byteorder:  little
byteswap:  False
type: <U70
None
-------------------------
[[['B1' 'Birmingham City Centre, Broad Street (east)' '52.4775396'
   '-1.894053']]

 [['B2' 'Birmingham City Centre, New Street' '52.4792602' '-1.8999756']]

 [['B3' 'Birmingham City Centre, Newhall Street' '52.4832071'
   '-1.9054204']]

 [['B4' 'Birmingham City Centre, Corporation Street (north)'
   '52.4775396' '-1.894053']]

 [['B5' 'Digbeth, Highgate, Lee Bank' '52.4734488' '-1.8871192']]

 [['B6' 'Aston, Witton' '52.506768' '-1.8806006']]

 [['B7' 'Nechells' '52.5023095' '-1.8605038']]

 [['B8' 'Washwood Heath, Ward End, Saltley' '52.4865451' '-1.8330255']]

 [['B9' 'Bordesley Green, Bordesley' '52.4778205' '-1.8487612']]

 [['B10' 'Small Heath' '52.4681262' '-1.8513874']]

 [['B11' 'Sparkhill, Sparkbrook, Tyseley' '52.4533095' '-1.8675068']]

 [['B

In [29]:
lat_list = []
lng_list = []
for i in range(21):
    lat_list.append(l1[i][0][2])
    lng_list.append(l1[i][0][3])
    i += 1
    
print(lat_list)
print(lng_list)

['52.4775396', '52.4792602', '52.4832071', '52.4775396', '52.4734488', '52.506768', '52.5023095', '52.4865451', '52.4778205', '52.4681262', '52.4533095', '52.4572246', '52.4347571', '52.4116475', '52.46882645', '52.4805954', '52.45960005', '52.4968772', '52.4996276', '52.5130035', '52.50767645']
['-1.894053', '-1.8999756', '-1.9054204', '-1.894053', '-1.8871192', '-1.8806006', '-1.8605038', '-1.8330255', '-1.8487612', '-1.8513874', '-1.8675068', '-1.8832067', '-1.87774549212102', '-1.8655917', '-1.92343540104981', '-1.918506', '-1.96040213627551', '-1.9380313', '-1.9068745', '-1.9067598', '-1.93312430441457']


In [30]:
#location_df = pd.DataFrame({ "Latitude" : lat_list, "Longitude" : lng_list})
#df2.reset_index(drop=True, inplace=True) 
df_ldn = df2.join(location_df)
df_ldn[['Latitude', 'Longitude']].astype('float64').dtypes
df_ldn.reset_index(drop=True, inplace=True)
df_ldn
#location_df

,Postcode district,Post town,Coverage,Local authority area,Address,Latitude,Longitude
0,E1,LONDON,"Eastern head district: Aldgate (part), Bishops...","Tower Hamlets, Hackney, City of London","Eastern head district: Aldgate (part), Bishops...",52.4775396,-1.894053
1,E1W,LONDON,"Wapping, St Katharine Docks, Stepney (part), S...",Tower Hamlets,"Wapping, St Katharine Docks, Stepney (part), S...",52.4792602,-1.8999756
2,E2,LONDON,"Bethnal Green district: Bethnal Green, Haggers...","Tower Hamlets, Hackney","Bethnal Green district: Bethnal Green, Haggers...",52.4832071,-1.9054204
3,E4,LONDON,"Chingford district: Chingford, Sewardstone, Hi...","Waltham Forest, Enfield, Epping Forest (Essex)","Chingford district: Chingford, Sewardstone, Hi...",52.4734488,-1.8871192
4,E5,LONDON,"Clapton district: Leyton (Part), Upper Clapton...","Hackney, Waltham Forest","Clapton district: Leyton (Part), Upper Clapton...",52.506768,-1.8806006
5,E6,LONDON,"East Ham district: East Ham, Beckton, Upton Pa...","Newham, Barking and Dagenham","East Ham district: East Ham, Beckton, Upton Pa...",52.5023095,-1.8605038
6,E8,LONDON,"Hackney district: Hackney Central, Dalston, Lo...",Hackney,"Hackney district: Hackney Central, Dalston, Lo...",52.4778205,-1.8487612
7,E9,LONDON,"Homerton district: Homerton, Hackney Wick, Sou...","Hackney, Tower Hamlets","Homerton district: Homerton, Hackney Wick, Sou...",52.4681262,-1.8513874
8,E10,LONDON,"Leyton district: Leyton, Temple Mills, Hackney...","Waltham Forest, Hackney","Leyton district: Leyton, Temple Mills, Hackney...",52.4533095,-1.8675068
9,E11,LONDON,"Leytonstone district: Leytonstone, Wanstead, A...","Waltham Forest, Redbridge","Leytonstone district: Leytonstone, Wanstead, A...",52.4572246,-1.8832067


In [31]:
len(lat_list)


21

In [32]:
df_ldn.drop([10], axis=0, inplace=True)

In [33]:
# create map of London UK using latitude and longitude values
latitude = 51.5074
longitude = -0.1278
map_ldn = folium.Map(location=[latitude, longitude], zoom_start=12.3)

# add markers to map
for lat, lng, pc, nbh in zip(df_ldn['Latitude'], df_ldn['Longitude'], df_ldn['Postcode district'], df_ldn['Coverage']):
    label = 'Postcode: {}\n, Area: {}\n'.format(pc, nbh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ldn)  
    
map_ldn

In [34]:
df_ldn.drop(['Post town','Local authority area','Address'], axis=1, inplace=True)
df_ldn

,Postcode district,Coverage,Latitude,Longitude
0,E1,"Eastern head district: Aldgate (part), Bishops...",52.4775396,-1.894053
1,E1W,"Wapping, St Katharine Docks, Stepney (part), S...",52.4792602,-1.8999756
2,E2,"Bethnal Green district: Bethnal Green, Haggers...",52.4832071,-1.9054204
3,E4,"Chingford district: Chingford, Sewardstone, Hi...",52.4734488,-1.8871192
4,E5,"Clapton district: Leyton (Part), Upper Clapton...",52.506768,-1.8806006
5,E6,"East Ham district: East Ham, Beckton, Upton Pa...",52.5023095,-1.8605038
6,E8,"Hackney district: Hackney Central, Dalston, Lo...",52.4778205,-1.8487612
7,E9,"Homerton district: Homerton, Hackney Wick, Sou...",52.4681262,-1.8513874
8,E10,"Leyton district: Leyton, Temple Mills, Hackney...",52.4533095,-1.8675068
9,E11,"Leytonstone district: Leytonstone, Wanstead, A...",52.4572246,-1.8832067


In [35]:
df_bham = df.drop(['Post town','Local authority area','Address'], axis=1, inplace=True)

KeyError: "['Post town' 'Local authority area' 'Address'] not found in axis"

In [37]:
df_bham = df.copy(deep=True)
df_bham

,Postcode district,Coverage,Latitude,Longitude
0,B1,"Birmingham City Centre, Broad Street (east)",52.4775396,-1.894053
1,B2,"Birmingham City Centre, New Street",52.4792602,-1.8999756
2,B3,"Birmingham City Centre, Newhall Street",52.4832071,-1.9054204
3,B4,"Birmingham City Centre, Corporation Street (no...",52.4775396,-1.894053
4,B5,"Digbeth, Highgate, Lee Bank",52.4734488,-1.8871192
5,B6,"Aston, Witton",52.506768,-1.8806006
6,B7,Nechells,52.5023095,-1.8605038
7,B8,"Washwood Heath, Ward End, Saltley",52.4865451,-1.8330255
8,B9,"Bordesley Green, Bordesley",52.4778205,-1.8487612
9,B10,Small Heath,52.4681262,-1.8513874


In [38]:
df = pd.concat([df_bham, df_ldn]).reset_index(drop=True)
df

,Postcode district,Coverage,Latitude,Longitude
0,B1,"Birmingham City Centre, Broad Street (east)",52.4775396,-1.894053
1,B2,"Birmingham City Centre, New Street",52.4792602,-1.8999756
2,B3,"Birmingham City Centre, Newhall Street",52.4832071,-1.9054204
3,B4,"Birmingham City Centre, Corporation Street (no...",52.4775396,-1.894053
4,B5,"Digbeth, Highgate, Lee Bank",52.4734488,-1.8871192
5,B6,"Aston, Witton",52.506768,-1.8806006
6,B7,Nechells,52.5023095,-1.8605038
7,B8,"Washwood Heath, Ward End, Saltley",52.4865451,-1.8330255
8,B9,"Bordesley Green, Bordesley",52.4778205,-1.8487612
9,B10,Small Heath,52.4681262,-1.8513874


In [39]:
radius = 1000
LIMIT = 50

In [42]:
# define Foursquare Credentials and Version
CLIENT_ID = 'EQKPCGSVJ5MFW1VBWSWL5QU5AEZ3PFOYXF2AAVK1B454WBQV' # your Foursquare ID
CLIENT_SECRET = '2AUYBTJ0SK0MUISM4LEITBXKZQAMBSZR24XYHC45OOZIKPFH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
total_venues = getNearbyVenues(names=df['Postcode district'], 
                                 latitudes=df['Latitude'], 
                                 longitudes=df['Longitude']
                                )

B1
B2
B3
B4
B5
B6
B7
B8
B9
B10
B11
B12
B13
B14
B15
B16
B17
B18
B19
B20
B21
B23
B24
B25
B26
B27
B28
B29
B30
B31
E1
E1W
E2
E4
E5
E6
E8
E9
E10
E11
E16
E17
E77
E98
EC1A
EC1M
EC1N
EC1P
EC1R
EC1V


In [49]:
print(total_venues.shape)
total_venues.sample(15)

(755, 7)


,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
420,B30,52.4273865,-1.9266395,Bournville Staff Shop (Cadbury World),52.427582,-1.929049,Candy Store
400,B28,52.4285007,-1.840403,Bull's Head,52.428863,-1.842198,Pub
244,B6,52.506768,-1.8806006,North Stand,52.509143,-1.885357,Soccer Stadium
487,E1W,52.4792602,-1.8999756,Purecraft Bar & Kitchen,52.480079,-1.901570,Bistro
736,EC1R,52.4285007,-1.840403,Bull's Head,52.428863,-1.842198,Pub
638,E6,52.5023095,-1.8605038,McDonald's,52.503938,-1.855178,Fast Food Restaurant
497,E1W,52.4792602,-1.8999756,Fumo,52.480666,-1.899239,Italian Restaurant
430,B31,52.4082313,-1.965252,Co-op Food,52.403841,-1.965552,Grocery Store
194,B4,52.4775396,-1.894053,Pret A Manger,52.477791,-1.898902,Sandwich Place
470,E1,52.4775396,-1.894053,200 Degrees Coffee,52.478801,-1.899521,Coffee Shop


In [50]:
total_venues.groupby('Postcode').count()

,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
B1,50,50,50,50,50,50
B10,6,6,6,6,6,6
B11,5,5,5,5,5,5
B12,16,16,16,16,16,16
B13,4,4,4,4,4,4
B14,5,5,5,5,5,5
B15,16,16,16,16,16,16
B16,11,11,11,11,11,11
B17,6,6,6,6,6,6


In [51]:
print('There are {} uniques categories.'.format(len(total_venues['Venue Category'].unique())))

There are 126 uniques categories.


In [52]:
# one hot encoding: creating columns for each unique category and using binary indicator variable for each row from toronto venues dataframe
total_onehot = pd.get_dummies(total_venues[['Venue Category']], prefix="", prefix_sep="")

# add postcode column back to dataframe
total_onehot['Postcode'] = total_venues['Postcode'] 

# move postcode column to the first column
fixed_columns = [total_onehot.columns[-1]] + list(total_onehot.columns[:-1])
total_onehot = total_onehot[fixed_columns]

total_onehot.head()

,Postcode,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bar,Beer Bar,Bistro,Bookstore,Botanical Garden,Bowling Alley,Brewery,Burger Joint,Bus Stop,Café,Candy Store,Casino,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Doner Restaurant,Donut Shop,Electronics Store,English Restaurant,Event Service,Farm,Farmers Market,Fast Food Restaurant,Food,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,History Museum,Hostel,Hotel,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Men's Store,Mexican Restaurant,Mini Golf,Monument / Landmark,Moroccan Restaurant,Multiplex,Music Venue,Nightclub,Noodle House,Optical Shop,Other Repair Shop,Pakistani Restaurant,Park,Pedestrian Plaza,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Plaza,Pool,Portuguese Restaurant,Post Office,Pub,Recreation Center,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Steakhouse,Supermarket,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wine Shop,Women's Store,Yoga Studio
0,B1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,B1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,B1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,B1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,B1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
total_onehot.shape


(755, 127)

In [54]:
total_grouped = total_onehot.groupby('Postcode').mean().reset_index()
total_grouped

,Postcode,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bar,Beer Bar,Bistro,Bookstore,Botanical Garden,Bowling Alley,Brewery,Burger Joint,Bus Stop,Café,Candy Store,Casino,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Doner Restaurant,Donut Shop,Electronics Store,English Restaurant,Event Service,Farm,Farmers Market,Fast Food Restaurant,Food,Food Truck,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,History Museum,Hostel,Hotel,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Men's Store,Mexican Restaurant,Mini Golf,Monument / Landmark,Moroccan Restaurant,Multiplex,Music Venue,Nightclub,Noodle House,Optical Shop,Other Repair Shop,Pakistani Restaurant,Park,Pedestrian Plaza,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Plaza,Pool,Portuguese Restaurant,Post Office,Pub,Recreation Center,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Steakhouse,Supermarket,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wine Shop,Women's Store,Yoga Studio
0,B1,0.000000,0.000000,0.000000,0.0200,0.00,0.00,0.0,0.00,0.000000,0.060000,0.00,0.00,0.04,0.0000,0.000000,0.00,0.08,0.000000,0.020000,0.000000,0.00,0.00,0.0000,0.060000,0.00,0.060000,0.00,0.00,0.0,0.02,0.000000,0.00,0.000000,0.04,0.000000,0.0,0.02,0.02,0.020000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.02,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.000000,0.020000,0.020000,0.04,0.04,0.000000,0.00,0.0,0.000000,0.000000,0.02,0.02,0.000000,0.000000,0.02,0.0000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.0,0.00,0.040000,0.000000,0.00,0.0000,0.020000,0.000000,0.020000,0.000000,0.000000,0.00,0.040000,0.04,0.000000,0.000000,0.000000,0.02,0.000000,0.0000,0.00,0.000000,0.02,0.00,0.020000,0.020000,0.000000,0.02,0.000000,0.000000,0.000000,0.02,0.02,0.000000,0.00,0.000000,0.0,0.00
1,B10,0.000000,0.000000,0.000000,0.0000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.000000,0.00,0.00,0.000000,0.166667,0.000000,0.00,0.00,0.0000,0.000000,0.00,0.000000,0.00,0.00,0.0,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.00,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.166667,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0000,0.166667,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.333333,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.166667,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00
2,B11,0.000000,0.000000,0.000000,0.0000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.0000,0.000000,0.00,0.000000,0.00,0.00,0.2,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.00,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.20,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.200000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.0,0.000

In [55]:
total_grouped.shape


(50, 127)

In [56]:
num_top_venues = 5

for pc in total_grouped['Postcode']:
    print("----"+pc+"----")
    temp = total_grouped[total_grouped['Postcode'] == pc].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----B1----
            venue  freq
0    Burger Joint  0.08
1  Clothing Store  0.06
2     Coffee Shop  0.06
3             Bar  0.06
4     Pizza Place  0.04


----B10----
                venue  freq
0      Sandwich Place  0.33
1  Turkish Restaurant  0.17
2   Indian Restaurant  0.17
3                Café  0.17
4                Park  0.17


----B11----
                  venue  freq
0  Pakistani Restaurant   0.4
1     Indian Restaurant   0.2
2     Convenience Store   0.2
3                   Gym   0.2
4   American Restaurant   0.0


----B12----
                  venue  freq
0     Indian Restaurant  0.31
1  Fast Food Restaurant  0.19
2                   Pub  0.06
3         Grocery Store  0.06
4  Pakistani Restaurant  0.06


----B13----
                venue  freq
0                 Pub  0.25
1  Athletics & Sports  0.25
2       Auto Workshop  0.25
3         Golf Course  0.25
4           Pet Store  0.00


----B14----
                  venue  freq
0           Gas Station   0.2
1    Chinese Restau

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postcode_venues_sorted = pd.DataFrame(columns=columns)
postcode_venues_sorted['Postcode'] = total_grouped['Postcode']

for ind in np.arange(total_grouped.shape[0]):
    postcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(total_grouped.iloc[ind, :], num_top_venues)

postcode_venues_sorted

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,B1,Burger Joint,Clothing Store,Bar,Coffee Shop,Pizza Place
1,B10,Sandwich Place,Indian Restaurant,Café,Park,Turkish Restaurant
2,B11,Pakistani Restaurant,Indian Restaurant,Convenience Store,Gym,Food
3,B12,Indian Restaurant,Fast Food Restaurant,Supermarket,Grocery Store,Moroccan Restaurant
4,B13,Pub,Athletics & Sports,Auto Workshop,Golf Course,Yoga Studio
5,B14,Fast Food Restaurant,Pharmacy,Gas Station,Grocery Store,Chinese Restaurant
6,B15,Hotel,Coffee Shop,Gastropub,Botanical Garden,Café
7,B16,Bar,Pub,Shopping Plaza,Music Venue,Restaurant
8,B17,Bakery,Deli / Bodega,Gym / Fitness Center,Coffee Shop,Thai Restaurant
9,B18,Light Rail Station,Fried Chicken Joint,Pizza Place,Vegetarian / Vegan Restaurant,Electronics Store


In [58]:
# set number of clusters
kclusters = 6

total_grouped_clustering = total_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(total_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 4, 4, 1, 1, 4, 4, 5], dtype=int32)

In [59]:
# add clustering labels
#postcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
postcode_venues_sorted['Cluster Labels'] = kmeans.labels_

total_merged = df

# merge total_grouped with total_data to add latitude/longitude for each neighborhood
total_merged = total_merged.join(postcode_venues_sorted.set_index('Postcode'), on='Postcode district')
total_merged = total_merged.dropna()
total_merged['Cluster Labels'] = total_merged['Cluster Labels'].astype('int64')
total_merged.head() # check the last columns!

,Postcode district,Coverage,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
0,B1,"Birmingham City Centre, Broad Street (east)",52.4775396,-1.894053,Burger Joint,Clothing Store,Bar,Coffee Shop,Pizza Place,4
1,B2,"Birmingham City Centre, New Street",52.4792602,-1.8999756,Coffee Shop,Bar,Pub,Italian Restaurant,Indian Restaurant,4
2,B3,"Birmingham City Centre, Newhall Street",52.4832071,-1.9054204,Pub,Indian Restaurant,Coffee Shop,Bar,Cocktail Bar,4
3,B4,"Birmingham City Centre, Corporation Street (no...",52.4775396,-1.894053,Burger Joint,Clothing Store,Bar,Coffee Shop,Pizza Place,4
4,B5,"Digbeth, Highgate, Lee Bank",52.4734488,-1.8871192,Pub,Music Venue,Bar,Café,Indian Restaurant,4


In [60]:
total_merged.shape


(49, 10)

In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(total_merged['Latitude'], total_merged['Longitude'], total_merged['Postcode district'], total_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [62]:
total_merged.groupby('Cluster Labels').count()

,Postcode district,Coverage,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Cluster Labels,,,,,,,,,
0,2,2,2,2,2,2,2,2,2
1,9,9,9,9,9,9,9,9,9
2,2,2,2,2,2,2,2,2,2
3,2,2,2,2,2,2,2,2,2
4,32,32,32,32,32,32,32,32,32
5,2,2,2,2,2,2,2,2,2


In [63]:
total_merged.loc[total_merged['Cluster Labels'] == 0, total_merged.columns[[0,1] + list(range(5, total_merged.shape[1]))]]

,Postcode district,Coverage,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
5,B6,"Aston, Witton",Pub,Soccer Field,Park,Yoga Studio,0
34,E5,"Clapton district: Leyton (Part), Upper Clapton...",Pub,Soccer Field,Park,Yoga Studio,0


In [64]:
total_merged.loc[total_merged['Cluster Labels'] == 1, total_merged.columns[[0,1] + list(range(5, total_merged.shape[1]))]]

,Postcode district,Coverage,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
6,B7,Nechells,Indian Restaurant,Hotel,Pizza Place,Noodle House,1
13,B14,"Kings Heath, Yardley Wood, Druids Heath, Hight...",Pharmacy,Gas Station,Grocery Store,Chinese Restaurant,1
14,B15,"Edgbaston,",Coffee Shop,Gastropub,Botanical Garden,Café,1
18,B19,"Lozells, Newtown, Birchfield",Soccer Stadium,Fast Food Restaurant,Yoga Studio,Food,1
19,B20,"Handsworth Wood, Handsworth, Birchfield",Pizza Place,Train Station,Yoga Studio,Food Truck,1
25,B27,Acocks Green,Hotel,Supermarket,Train Station,Mexican Restaurant,1
27,B29,"Selly Oak, Bournbrook, Selly Park, Weoley Cast...",Supermarket,Fried Chicken Joint,Liquor Store,Fast Food Restaurant,1
35,E6,"East Ham district: East Ham, Beckton, Upton Pa...",Indian Restaurant,Hotel,Pizza Place,Noodle House,1
49,EC1V,"Finsbury (east), Moorfields Eye Hospital",Supermarket,Fried Chicken Joint,Liquor Store,Fast Food Restaurant,1


In [65]:
total_merged.loc[total_merged['Cluster Labels'] == 3, total_merged.columns[[0,1] + list(range(5, total_merged.shape[1]))]]

,Postcode district,Coverage,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
8,B9,"Bordesley Green, Bordesley",Automotive Shop,Fried Chicken Joint,Department Store,Dessert Shop,3
36,E8,"Hackney district: Hackney Central, Dalston, Lo...",Automotive Shop,Fried Chicken Joint,Department Store,Dessert Shop,3


In [66]:
total_merged.loc[total_merged['Cluster Labels'] == 4, total_merged.columns[[0,1] + list(range(5, total_merged.shape[1]))]]

,Postcode district,Coverage,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
0,B1,"Birmingham City Centre, Broad Street (east)",Clothing Store,Bar,Coffee Shop,Pizza Place,4
1,B2,"Birmingham City Centre, New Street",Bar,Pub,Italian Restaurant,Indian Restaurant,4
2,B3,"Birmingham City Centre, Newhall Street",Indian Restaurant,Coffee Shop,Bar,Cocktail Bar,4
3,B4,"Birmingham City Centre, Corporation Street (no...",Clothing Store,Bar,Coffee Shop,Pizza Place,4
4,B5,"Digbeth, Highgate, Lee Bank",Music Venue,Bar,Café,Indian Restaurant,4
7,B8,"Washwood Heath, Ward End, Saltley",Waste Facility,Asian Restaurant,Construction & Landscaping,Electronics Store,4
9,B10,Small Heath,Indian Restaurant,Café,Park,Turkish Restaurant,4
10,B11,"Sparkhill, Sparkbrook, Tyseley",Indian Restaurant,Convenience Store,Gym,Food,4
11,B12,"Balsall Heath, Sparkbrook, Highgate",Fast Food Restaurant,Supermarket,Grocery Store,Moroccan Restaurant,4
12,B13,"Moseley, Billesley",Athletics & Sports,Auto Workshop,Golf Course,Yoga Studio,4


In [67]:
total_merged.loc[total_merged['Cluster Labels'] == 5, total_merged.columns[[0,1] + list(range(5, total_merged.shape[1]))]]

,Postcode district,Coverage,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
17,B18,"Winson Green, Hockley",Fried Chicken Joint,Pizza Place,Vegetarian / Vegan Restaurant,Electronics Store,5
41,E17,"Walthamstow district: Walthamstow, Upper Walth...",Fried Chicken Joint,Pizza Place,Vegetarian / Vegan Restaurant,Electronics Store,5
